In [164]:
# -*- coding: utf-8 -*-
"""
Created on Mon Oct 19 12:48:59 2020

@author: mathemacode

Machine learning modeling of ERAU PSF data after
feature engineering done to create "ML_frame.csv".

Data file built in ./R/all_numbers_merge_data.R

"""

import pandas as pd
import shap
import eli5
import os
from eli5.sklearn import PermutationImportance
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from xgboost import XGBRegressor


os.getcwd()
os.chdir("C:\\Users\\dell\\Documents\\GitHub\\PSF_ERAU")

# Import data
df = pd.read_csv('./data/ml/ML_frame.csv')

df.head()

,id,zip,zip_count,number_removals,number_placements,number_participants,case_duration_yrs,number_caregivers,age_child,avg_age_caregiver,avg_gross_income_zip,first_placement,multiple_removals
0,6017341000,34479,1,4,17,25,4.4,8,13.6,48.4,42992,1,1
1,9123872000,32091,68,1,1,3,0.6,2,15.4,42.6,48438,2,0
2,1152740000,32780,1,4,25,10,2.1,4,16.1,37.0,170585,1,1
3,10101546999,32696,42,1,1,8,1.2,3,14.4,47.1,44626,3,0
4,3544513075,32601,82,2,5,15,1.9,3,13.3,47.9,54063,2,1


In [165]:
# Normalize df
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df)
df_norm = pd.DataFrame(np_scaled)

df_norm.columns = df.columns
df_norm.head()

,id,zip,zip_count,number_removals,number_placements,number_participants,case_duration_yrs,number_caregivers,age_child,avg_age_caregiver,avg_gross_income_zip,first_placement,multiple_removals
0,0.595406,0.827528,0.000000,0.214286,0.347826,0.666667,0.619718,0.500000,0.600000,0.514894,0.063069,0.0,1.0
1,0.902792,0.027796,0.475177,0.000000,0.000000,0.055556,0.084507,0.071429,0.680000,0.432624,0.071524,0.5,0.0
2,0.114062,0.258540,0.000000,0.214286,0.521739,0.250000,0.295775,0.214286,0.711111,0.353191,0.261153,0.0,1.0
3,0.999532,0.230409,0.290780,0.000000,0.000000,0.194444,0.169014,0.142857,0.635556,0.496454,0.065606,1.0,0.0
4,0.350724,0.198593,0.574468,0.071429,0.086957,0.388889,0.267606,0.142857,0.586667,0.507801,0.080256,0.5,1.0


In [166]:
details = df.describe()
details_nocount = details.iloc[1:]
details

,id,zip,zip_count,number_removals,number_placements,number_participants,case_duration_yrs,number_caregivers,age_child,avg_age_caregiver,avg_gross_income_zip,first_placement,multiple_removals
count,1.597000e+03,1597.000000,1597.000000,1597.000000,1597.000000,1597.000000,1597.000000,1597.000000,1597.000000,1597.000000,1597.000000,1597.000000,1597.000000
mean,8.765894e+09,32460.839699,71.609267,1.442079,2.922354,11.126487,1.269505,3.793363,7.725924,39.553914,67081.168441,2.325611,0.219787
std,2.716299e+09,449.840720,42.443064,1.219077,3.428866,5.631012,1.184323,1.666004,5.198069,8.819033,67350.494657,0.875267,0.414232
min,6.120000e+02,32008.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.100000,12.100000,2367.000000,1.000000,0.000000
25%,9.734560e+09,32059.000000,37.000000,1.000000,1.000000,7.000000,0.300000,3.000000,3.200000,33.600000,25088.000000,1.000000,0.000000
50%,1.010149e+10,32607.000000,68.000000,1.000000,2.000000,10.000000,1.100000,4.000000,7.000000,39.100000,51412.000000,3.000000,0.000000
75%,1.010264e+10,32626.000000,118.000000,1.000000,3.000000,14.000000,1.900000,5.000000,11.700000,44.600000,72355.000000,3.000000,0.000000
max,1.010628e+10,34994.000000,142.000000,15.000000,47.000000,37.000000,7.100000,15.000000,22.600000,82.600000,646502.000000,3.000000,1.000000


In [167]:
# Use these rows for prediction:
features = ['zip', 'zip_count', 'number_participants',
            'case_duration_yrs', 'number_caregivers',
            'age_child', 'avg_age_caregiver',
            'avg_gross_income_zip', 'first_placement']
X = df_norm[features]

# What to predict:
y = df_norm.multiple_removals

# Train/test split
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

In [168]:
# Random Forest
rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(train_X, train_y)

rf_predictions = rf_model.predict(val_X)
print("\nRandom Forest Mean Absolute Error: \n", 
      mean_absolute_error(val_y, rf_predictions))

perm = PermutationImportance(rf_model, random_state=1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist())

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



Random Forest Mean Absolute Error: 
 0.1825


Weight,Feature
0.0600 ± 0.0190,number_participants
0.0485 ± 0.0117,case_duration_yrs
0.0380 ± 0.0215,avg_gross_income_zip
0.0320 ± 0.0231,age_child
0.0290 ± 0.0081,number_caregivers
0.0285 ± 0.0154,first_placement
0.0200 ± 0.0126,zip_count
0.0165 ± 0.0051,avg_age_caregiver
0.0085 ± 0.0103,zip


In [169]:
# XGBoost
xg_model = xgb.XGBClassifier(n_estimators=500, learning_rate=0.05)
xg_model.fit(train_X, train_y,
             early_stopping_rounds=5, 
             eval_set=[(val_X, val_y)], 
             verbose=False)

xg_predictions = xg_model.predict(val_X)
print("\nXGBoost Mean Absolute Error: \n",
      str(mean_absolute_error(xg_predictions, val_y)))

perm = PermutationImportance(xg_model, random_state=1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist())


XGBoost Mean Absolute Error: 
 0.25


Weight,Feature
0.0435 ± 0.0238,number_participants
0.0320 ± 0.0058,first_placement
0.0225 ± 0.0045,avg_age_caregiver
0.0225 ± 0.0207,case_duration_yrs
0.0160 ± 0.0172,age_child
0.0095 ± 0.0058,number_caregivers
0.0065 ± 0.0051,zip
0.0025 ± 0.0122,avg_gross_income_zip
0 ± 0.0000,zip_count


In [173]:
# Neural Network
X, y = make_classification(n_samples=200, random_state=1)

NN = MLPClassifier(random_state=1, max_iter=2000).fit(train_X, train_y)

NN_predictions = NN.predict(val_X)
print("\nNeural Network Mean Absolute Error: \n",
      str(mean_absolute_error(NN_predictions, val_y)))

# regr.score(val_X, val_y)
# regr.predict(val_X[:3])

perm = PermutationImportance(NN, random_state=1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist())


Neural Network Mean Absolute Error: 
 0.2275


Weight,Feature
0.0485 ± 0.0196,first_placement
0.0245 ± 0.0344,number_participants
0.0235 ± 0.0225,number_caregivers
0.0230 ± 0.0139,zip
0.0175 ± 0.0182,case_duration_yrs
0.0125 ± 0.0182,age_child
0.0110 ± 0.0229,zip_count
0.0085 ± 0.0121,avg_gross_income_zip
0.0045 ± 0.0097,avg_age_caregiver
